In [53]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.impute import (SimpleImputer,KNNImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, normalize
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [54]:
def fill_missing_values_KNN(X, X_test): 
    #normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    
    X_norma = (X-X_ave)/X_std
    X_test_norma = (X_test-X_ave)/X_std

    imputer = KNNImputer(missing_values=np.nan, n_neighbors=75, weights = 'distance')
    X_norma_fixed = imputer.fit_transform(X_norma)
    X_test_norma_fixed = imputer.fit_transform(X_test_norma)

    return X_norma_fixed, X_test_norma_fixed

# def fill_missing_values_mean(X, X_test):
#     imp = SimpleImputer(missing_values=np.nan, strategy='median')
#     imp.fit(X)
#     X = imp.transform(X)
#     imp.fit(X_test)
#     X_test = imp.transform(X_test)
#     return X, X_test

def fill_missing_values_median(X):
    imp = SimpleImputer(missing_values=np.nan, strategy='median')
    imp.fit(X)
    X = imp.transform(X)
    return X

def remove_outliers_XY(X, y, y_additional_degree):
#     print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    
    Y = y
    for degree in y_additional_degree:
        Y = np.c_[Y, y**degree]
        
    Z = np.c_[X, Y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, Y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
#     X , y = X[mask, :], y[mask]
#     print("IsolationForest-Traing data shape after removed: {}".format(X.shape))
    return mask



def find_missing_value_and_move_outliers(X, y, X_test,y_additional_degree):
    print("Traing data shape before impute and outlier remove: {}".format(X.shape))
    print("Testing data shape before impute and outlier remove: {}".format(X_test.shape))
    X_nan = np.isnan(X)  
    X_median = fill_missing_values_median(X)
    
    IstInLiers = remove_outliers_XY(X_median, y, y_additional_degree)
    X_median = X_median[IstInLiers, :]
    X_nan = X_nan[IstInLiers, :]
    y = y[IstInLiers]
    
    X_median[X_nan] = np.nan
    
    X_KNN,X_test_KNN = fill_missing_values_KNN(X_median, X_test)
    print("Traing data shape after impute and outlier remove: {}".format(X_KNN.shape))
    print("Testing data shape after impute and outlier remove: {}".format(X_test_KNN.shape))
    
    return X_KNN,X_test_KNN,y



In [55]:
def select_features(X, y, X_test, feature_num=100):
    scaler = StandardScaler().fit(X, y)
    X = scaler.transform(X)
    X_test = scaler.transform(X_test)
    
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=75, random_state=1)
#     rf = ExtraTreesRegressor(n_jobs=-1, max_depth=None, n_estimators=180, random_state=1, min_samples_split=3, max_features=None)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    
    return X, X_test
def auto_feature_extraction(X, y, X_test):
    fsel = FeatureSelector(verbose=1)
    
    feature_names = [str(i) for i in range(0,X.shape[1])]
    X_pd = pd.DataFrame(X, columns=feature_names)
    new_X = fsel.fit_transform(X_pd, y)
    print("len of columns:", len(new_X.columns))
    print(new_X.columns)
    index_chosen = [int(new_X.columns[i]) for i in range(0,len(new_X.columns))]
    return X[:,index_chosen],X_test[:,index_chosen]

In [56]:
X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Imputation of Missing Values
* [Reference](https://scikit-learn.org/stable/modules/impute.html)
* We use median of column instead of mean

In [57]:
# X_train_missing_indices = X_train[X_train==np]
# X_train, X_test = fill_missing_values(X_train, X_test)
# print(X_train.shape)
# print(X_test.shape)

## 2. Outlier Detection
* [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)

In [58]:
# X_train,y_train = remove_outliers(X_train,y_train)
X_train_processed, X_test_processed, y_processed = find_missing_value_and_move_outliers(X_train, y_train, X_test,[0.5,1.5,2,-1])

Traing data shape before impute and outlier remove: (1212, 832)
Testing data shape before impute and outlier remove: (776, 832)


<ipython-input-54-7fbf307def4b>:6: RuntimeWarning: invalid value encountered in true_divide
  X_norma = (X-X_ave)/X_std
<ipython-input-54-7fbf307def4b>:7: RuntimeWarning: invalid value encountered in true_divide
  X_test_norma = (X_test-X_ave)/X_std


Traing data shape after impute and outlier remove: (1198, 828)
Testing data shape after impute and outlier remove: (776, 828)


## 3. Feature Selection

In [59]:
def auto_feature_extraction(X, y, X_test):
    fsel = FeatureSelector(verbose=1)
    
    feature_names = [str(i) for i in range(0,X.shape[1])]
    X_pd = pd.DataFrame(X, columns=feature_names)
    new_X = fsel.fit_transform(X_pd, y)
    print("len of columns:", len(new_X.columns))
    print(new_X.columns)
    index_chosen = [int(new_X.columns[i]) for i in range(0,len(new_X.columns))]
    return X[:,index_chosen],X_test[:,index_chosen]

In [60]:
X_feature_from_batch1, X_test_feature_from_batch1 = X_train_processed, X_test_processed

In [61]:
from autofeat import FeatureSelector, AutoFeatRegressor
split_num = 20
split_size = int(X_test_feature_from_batch1.shape[1]/split_num)
first_flag = 1
for i in range(0,split_num):
    split_index = [j for j in range(i*split_size,min((i+1)*split_size,X_test_feature_from_batch1.shape[1]))]
    X_feature_subset = X_feature_from_batch1[:,split_index]
    X_test__feature_subset = X_test_feature_from_batch1[:,split_index]
    X_feature_subset, X_test__feature_subset = auto_feature_extraction(X_feature_subset, y_processed, X_test__feature_subset)

    if(first_flag):
        first_flag = 0
        X_feature_from_batch2 = X_feature_subset
        X_test_feature_from_batch2 = X_test__feature_subset
    else:

        X_feature_from_batch2 = np.hstack((X_feature_from_batch2,X_feature_subset))
        X_test_feature_from_batch2 = np.hstack((X_test_feature_from_batch2,X_test__feature_subset))
print("shape of X_feature_from_batch2:",X_test_feature_from_batch2.shape)
print("shape of X_test_feature_from_batch2:",X_test_feature_from_batch2.shape)

[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 8 features after 5 feature selection runs
[featsel] 8 features after correlation filtering
[featsel] 8 features after noise filtering
len of columns: 8
Index(['7', '35', '39', '30', '22', '10', '12', '25'], dtype='object')
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 9 features after 5 feature selection runs
[featsel] 9 features after correlation filtering
[featsel] 8 features after noise filtering
len of columns: 8
Index(['0', '33', '11', '26', '21', '40', '27', '1'], dtype='object')
[featsel] Scaling data...done.
[featsel] Feature selection run 1/5
[featsel] Feature selection run 2/5
[featsel] 

[featsel] Feature selection run 2/5
[featsel] Feature selection run 3/5
[featsel] Feature selection run 4/5
[featsel] Feature selection run 5/5
[featsel] 10 features after 5 feature selection runs
[featsel] 10 features after correlation filtering
[featsel] 9 features after noise filtering
len of columns: 9
Index(['6', '2', '20', '40', '3', '9', '17', '18', '31'], dtype='object')
shape of X_feature_from_batch2: (776, 149)
shape of X_test_feature_from_batch2: (776, 149)


In [62]:
X_feature_from_batch3, X_test_feature_from_batch3 = select_features(X_feature_from_batch2, y_processed, X_test_feature_from_batch2,feature_num = 60)
print("shape of X_feature_from_batch3: ", X_feature_from_batch3.shape)
print("shape of X_test_feature_from_batch3: ", X_test_feature_from_batch3.shape)

shape of X_feature_from_batch3:  (1198, 60)
shape of X_test_feature_from_batch3:  (776, 60)


## 4. Extra Tree

In [82]:



def custom_r2(prediction, train_data):
    """Regular r2 cost function returned as a tuple to be used with lgb"""
    labels = train_data.get_label()
    return 'r2', r2_score(labels, prediction), True

def fit_model_and_pred(degree, X_train, y_train, X_val, y_val, X_test):
   
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'num_leaves': 1700,
        'learning_rate': 0.025,
        'max_depth': 11,
        'n_estimators': 1000,
        'min_child_weight': 1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'feature_fraction': 0.5,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0,
        'num_iterations':600,
    }
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=500,
                    feval=custom_r2,
                    valid_sets={lgb_train, lgb_eval},
                    early_stopping_rounds=100
                   )

    y_pred = gbm.predict(X_test) 
    
    return y_pred

def train_k_fold(X, y, fold_num=10):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=True)
    kf.get_n_splits(X)
    test_score = 0.0
    
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        y_pred = fit_model_and_pred(1, X_train, y_train, X_val, y_val, X_val)
        score = r2_score(y_val, y_pred)

        print('The obtained validation r2 score is : ',score)
        test_score += score
    print("Validation score: %f"%(test_score/fold_num))
    
def train_k_fold_predict(X, y,X_test, fold_num=10):
    kf = KFold(n_splits=fold_num, random_state=None, shuffle=True)
    kf.get_n_splits(X)
    test_score = 0.0
    y_test_predict = np.zeros(X_test.shape[0])
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]
        score = r2_score(y_val, fit_model_and_pred(1, X_train, y_train, X_val, y_val, X_val))
        test_score += score
        
        y_pred = fit_model_and_pred(1, X_train, y_train, X_val, y_val, X_test)
        y_test_predict += y_pred
        
    print("Validation score: %f"%(test_score/fold_num))
    return y_test_predict/fold_num

# 以下是树训练模型
# def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test):
#     model = ExtraTreesRegressor(n_jobs=1, max_depth=None, n_estimators=195, random_state=0, min_samples_split=2, max_features=None)

#     model.fit(X_train, y_train)
    
#     y_val_pred = model.predict(X_val)
#     score = r2_score(y_val, y_val_pred)
    
#     y_pred = model.predict(X_test) 
    
#     return score, y_pred


# def train_k_fold(X, y, fold_num=10):
#     kf = KFold(n_splits=fold_num, random_state=None, shuffle=False)
#     kf.get_n_splits(X)
#     test_score = 0.0
    
#     for train_index, test_index in kf.split(X):
#         X_train, X_val = X[train_index], X[test_index]
#         y_train, y_val = y[train_index], y[test_index]

#         y_pred = fit_model_and_pred(1, X_train, y_train, X_val, y_val, X_val)
#         score = r2_score(y_val, y_pred)

#         print('The obtained validation r2 score is : ',score)
#         test_score += score
#     print("Validation score: %f"%(test_score/fold_num))
    

# def train_k_fold_predict(X, y, X_test, fold_num=10):
#     kf = KFold(n_splits=fold_num)
#     kf.get_n_splits(X)
#     y_test_predict = np.zeros(X_test.shape[0])
#     cnt = 0
#     val_score = 0.0
    
#     for train_index, test_index in kf.split(X):
#         X_train, X_val = X[train_index], X[test_index]
#         y_train, y_val = y[train_index], y[test_index]

#         score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test)
#         val_score += score
#         if(score > 0.7):
#             y_test_predict += y_pred
#             cnt += 1
#     return val_score/fold_num, y_test_predict/cnt


In [83]:
train_k_fold(X_feature_from_batch3,y_processed) #Knn with std and feature num = 100

/home/yang/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/yang/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/yang/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000818 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l2: 92.0044	training's

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[70]	training's l2: 22.7636	training's r2: 0.759828	valid_0's l2: 46.4923	valid_0's r2: 0.49435
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[71]	training's l2: 22.4561	training's r2: 0.763073	valid_0's l2: 46.356	valid_0's r2: 0.495833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[72]	training's l2: 22.1692	training's r2: 0.7661	valid_0's l2: 46.1731	valid_0's r2: 0.497821
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[73]	training's l2: 21.9011	training's r2: 0.768928	valid_0's l2: 45.945	valid_0's r2: 0.500303
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	training's l2: 21.6216	training's r2: 0.771877	valid_0's l2: 45.8073	valid_0's r2: 0.5018
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	training's l2: 21.3386	training's r2: 0.774863	valid_0's l2: 45.674

[173]	training's l2: 9.26082	training's r2: 0.902292	valid_0's l2: 40.8774	valid_0's r2: 0.555417
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[174]	training's l2: 9.2006	training's r2: 0.902927	valid_0's l2: 40.8225	valid_0's r2: 0.556014
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[175]	training's l2: 9.15431	training's r2: 0.903416	valid_0's l2: 40.8039	valid_0's r2: 0.556217
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[176]	training's l2: 9.10056	training's r2: 0.903983	valid_0's l2: 40.7759	valid_0's r2: 0.556521
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[177]	training's l2: 9.03819	training's r2: 0.904641	valid_0's l2: 40.843	valid_0's r2: 0.555791
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[178]	training's l2: 8.97971	training's r2: 0.905258	valid_0's l2: 40.8141	valid_0's r2: 0.556106
[LightGBM] [Warning] No further splits 

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[276]	training's l2: 5.42105	training's r2: 0.942804	valid_0's l2: 39.3891	valid_0's r2: 0.571604
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[277]	training's l2: 5.39216	training's r2: 0.943109	valid_0's l2: 39.321	valid_0's r2: 0.572345
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[278]	training's l2: 5.35804	training's r2: 0.943469	valid_0's l2: 39.3043	valid_0's r2: 0.572526
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[279]	training's l2: 5.34089	training's r2: 0.94365	valid_0's l2: 39.3165	valid_0's r2: 0.572394
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[280]	training's l2: 5.32392	training's r2: 0.943829	valid_0's l2: 39.3373	valid_0's r2: 0.572168
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[281]	training's l2: 5.29615	training's r2: 0.944122	valid_0's

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[79]	training's l2: 21.1277	training's r2: 0.77514	valid_1's l2: 39.4061	valid_1's r2: 0.60376
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[80]	training's l2: 20.8723	training's r2: 0.777858	valid_1's l2: 39.3602	valid_1's r2: 0.604222
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[81]	training's l2: 20.6503	training's r2: 0.780221	valid_1's l2: 39.2944	valid_1's r2: 0.604884
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[82]	training's l2: 20.403	training's r2: 0.782853	valid_1's l2: 39.136	valid_1's r2: 0.606476
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[83]	training's l2: 20.1723	training's r2: 0.785309	valid_1's l2: 38.9782	valid_1's r2: 0.608063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[84]	training's l2: 19.955	training's r2: 0.787621	valid_1's l2: 38.8

[176]	training's l2: 9.57728	training's r2: 0.89807	valid_1's l2: 32.9777	valid_1's r2: 0.6684
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[177]	training's l2: 9.52702	training's r2: 0.898605	valid_1's l2: 33.0084	valid_1's r2: 0.668091
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[178]	training's l2: 9.47248	training's r2: 0.899186	valid_1's l2: 32.9738	valid_1's r2: 0.668439
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[179]	training's l2: 9.42508	training's r2: 0.89969	valid_1's l2: 32.9454	valid_1's r2: 0.668725
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[180]	training's l2: 9.37405	training's r2: 0.900233	valid_1's l2: 32.8843	valid_1's r2: 0.669339
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[181]	training's l2: 9.33825	training's r2: 0.900614	valid_1's l2: 32.8529	valid_1's r2: 0.669655
[LightGBM] [Warning] No further splits wi

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[272]	training's l2: 5.97998	training's r2: 0.936356	valid_1's l2: 31.9065	valid_1's r2: 0.679171
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[273]	training's l2: 5.9513	training's r2: 0.936661	valid_1's l2: 31.8992	valid_1's r2: 0.679245
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[274]	training's l2: 5.91735	training's r2: 0.937022	valid_1's l2: 31.8778	valid_1's r2: 0.67946
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[275]	training's l2: 5.8956	training's r2: 0.937254	valid_1's l2: 31.8724	valid_1's r2: 0.679514
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[276]	training's l2: 5.86103	training's r2: 0.937622	valid_1's l2: 31.8693	valid_1's r2: 0.679545
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[277]	training's l2: 5.83154	training's r2: 0.937936	valid_1's 

[359]	training's l2: 4.05791	training's r2: 0.956812	valid_1's l2: 31.4583	valid_1's r2: 0.683678
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[360]	training's l2: 4.04234	training's r2: 0.956978	valid_1's l2: 31.4532	valid_1's r2: 0.683729
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[361]	training's l2: 4.02956	training's r2: 0.957114	valid_1's l2: 31.4402	valid_1's r2: 0.683859
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[362]	training's l2: 4.01121	training's r2: 0.957309	valid_1's l2: 31.4454	valid_1's r2: 0.683808
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[363]	training's l2: 3.9917	training's r2: 0.957517	valid_1's l2: 31.4355	valid_1's r2: 0.683907
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[364]	training's l2: 3.97382	training's r2: 0.957707	valid_1's l2: 31.4219	valid_1's r2: 0.684043
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[457]	training's l2: 2.84427	training's r2: 0.969729	valid_1's l2: 31.5466	valid_1's r2: 0.68279
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[458]	training's l2: 2.83501	training's r2: 0.969827	valid_1's l2: 31.5705	valid_1's r2: 0.68255
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[459]	training's l2: 2.82777	training's r2: 0.969904	valid_1's l2: 31.5501	valid_1's r2: 0.682755
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[460]	training's l2: 2.81935	training's r2: 0.969994	valid_1's l2: 31.5438	valid_1's r2: 0.682818
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[461]	training's l2: 2.81117	training's r2: 0.970081	valid_1's l2: 31.5377	valid_1's r2: 0.68288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[462]	training's l2: 2.80245	training's r2: 0.970174	valid_1's 

In [84]:
Y_test_pred = train_k_fold_predict(X_feature_from_batch3,y_processed,X_test_feature_from_batch3)
final_res = np.vstack((indices_test, Y_test_pred)).T

/home/yang/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/yang/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/home/yang/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000771 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=0.8 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's l2: 90.7637	training's

[61]	training's l2: 25.8275	training's r2: 0.723825	valid_0's l2: 46.4633	valid_0's r2: 0.5505
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[62]	training's l2: 25.4454	training's r2: 0.727911	valid_0's l2: 46.1236	valid_0's r2: 0.553787
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[63]	training's l2: 25.0912	training's r2: 0.731699	valid_0's l2: 45.9315	valid_0's r2: 0.555645
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[64]	training's l2: 24.7166	training's r2: 0.735704	valid_0's l2: 45.5449	valid_0's r2: 0.559385
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[65]	training's l2: 24.3841	training's r2: 0.73926	valid_0's l2: 45.1994	valid_0's r2: 0.562727
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[66]	training's l2: 24.0778	training's r2: 0.742534	valid_0's l2: 44.9722	valid_0's r2: 0.564925
[LightGBM] [Warning] No further splits with po

[147]	training's l2: 11.0712	training's r2: 0.881615	valid_0's l2: 36.4285	valid_0's r2: 0.647579
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[148]	training's l2: 10.9782	training's r2: 0.88261	valid_0's l2: 36.2947	valid_0's r2: 0.648874
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[149]	training's l2: 10.8896	training's r2: 0.883557	valid_0's l2: 36.2519	valid_0's r2: 0.649288
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[150]	training's l2: 10.8112	training's r2: 0.884396	valid_0's l2: 36.2014	valid_0's r2: 0.649776
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[151]	training's l2: 10.7345	training's r2: 0.885216	valid_0's l2: 36.1025	valid_0's r2: 0.650734
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[152]	training's l2: 10.6604	training's r2: 0.886007	valid_0's l2: 36.0925	valid_0's r2: 0.650831
[LightGBM] [Warning] No further splits

[242]	training's l2: 6.66185	training's r2: 0.928765	valid_0's l2: 34.0449	valid_0's r2: 0.670639
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[243]	training's l2: 6.62121	training's r2: 0.929199	valid_0's l2: 34.0199	valid_0's r2: 0.670881
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[244]	training's l2: 6.5906	training's r2: 0.929526	valid_0's l2: 33.9678	valid_0's r2: 0.671385
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[245]	training's l2: 6.55329	training's r2: 0.929925	valid_0's l2: 33.9379	valid_0's r2: 0.671674
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[246]	training's l2: 6.53327	training's r2: 0.930139	valid_0's l2: 33.9575	valid_0's r2: 0.671485
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[247]	training's l2: 6.49549	training's r2: 0.930543	valid_0's l2: 33.9337	valid_0's r2: 0.671715
[LightGBM] [Warning] No further splits

...

[245]	training's l2: 6.48004	training's r2: 0.931588	valid_1's l2: 32.7339	valid_1's r2: 0.645699
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[246]	training's l2: 6.45527	training's r2: 0.931849	valid_1's l2: 32.7036	valid_1's r2: 0.646028
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[247]	training's l2: 6.42901	training's r2: 0.932126	valid_1's l2: 32.663	valid_1's r2: 0.646467
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[248]	training's l2: 6.40246	training's r2: 0.932407	valid_1's l2: 32.6457	valid_1's r2: 0.646654
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[249]	training's l2: 6.36739	training's r2: 0.932777	valid_1's l2: 32.5891	valid_1's r2: 0.647267
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[250]	training's l2: 6.34442	training's r2: 0.933019	valid_1's l2: 32.5812	valid_1's r2: 0.647352
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[337]	training's l2: 4.29311	training's r2: 0.954676	valid_1's l2: 32.1312	valid_1's r2: 0.652223
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[338]	training's l2: 4.27659	training's r2: 0.95485	valid_1's l2: 32.1364	valid_1's r2: 0.652167
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[339]	training's l2: 4.25711	training's r2: 0.955056	valid_1's l2: 32.1609	valid_1's r2: 0.651902
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[340]	training's l2: 4.24245	training's r2: 0.955211	valid_1's l2: 32.1361	valid_1's r2: 0.65217
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[341]	training's l2: 4.23092	training's r2: 0.955333	valid_1's l2: 32.1251	valid_1's r2: 0.652289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[342]	training's l2: 4.21829	training's r2: 0.955466	valid_1's

[427]	training's l2: 3.03403	training's r2: 0.967969	valid_1's l2: 31.9348	valid_1's r2: 0.654349
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[428]	training's l2: 3.02702	training's r2: 0.968043	valid_1's l2: 31.9342	valid_1's r2: 0.654355
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[429]	training's l2: 3.01381	training's r2: 0.968182	valid_1's l2: 31.9203	valid_1's r2: 0.654506
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[430]	training's l2: 3.00454	training's r2: 0.96828	valid_1's l2: 31.899	valid_1's r2: 0.654736
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[431]	training's l2: 2.99615	training's r2: 0.968368	valid_1's l2: 31.9051	valid_1's r2: 0.65467
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[432]	training's l2: 2.98063	training's r2: 0.968532	valid_1's l2: 31.8974	valid_1's r2: 0.654754
[LightGBM] [Warning] No further splits w

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[518]	training's l2: 2.26662	training's r2: 0.97607	valid_1's l2: 31.6162	valid_1's r2: 0.657797
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[519]	training's l2: 2.26259	training's r2: 0.976113	valid_1's l2: 31.6052	valid_1's r2: 0.657917
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[520]	training's l2: 2.25262	training's r2: 0.976218	valid_1's l2: 31.6271	valid_1's r2: 0.657679
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[521]	training's l2: 2.24279	training's r2: 0.976322	valid_1's l2: 31.6277	valid_1's r2: 0.657673
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[522]	training's l2: 2.23572	training's r2: 0.976397	valid_1's l2: 31.6341	valid_1's r2: 0.657604
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[523]	training's l2: 2.22822	training's r2: 0.976476	valid_1'

Validation score: 0.639078


In [85]:
pd.DataFrame(final_res).to_csv("our_result.csv", header = ["id", "y"], index=False)